### 동영상-차량 이동 추적

In [1]:
# 루카스-카나데(Lucas-Kanade) 알고리즘
#   광학 흐름, 이웃하는 픽셀이 비슷하게 움직인다고 가정
# 이웃하는 픽셀은 비슷한 움직임을 갖는다고 생각하고 광학 흐름을 파악
# 3 x 3 patch를 사용하여 움직임 계산

import numpy as np
import cv2 as cv

cap=cv.VideoCapture('c:/workspace3/data/video/slow_traffic_small.mp4')

feature_params=dict(maxCorners=100,qualityLevel=0.3,minDistance=7,blockSize=7)
lk_params=dict(winSize=(15,15),maxLevel=2,criteria=(cv.TERM_CRITERIA_EPS|cv.TERM_CRITERIA_COUNT,10,0.03))
color=np.random.randint(0,255,(100,3))

ret,old_frame=cap.read()        # 첫 프레임
old_gray=cv.cvtColor(old_frame,cv.COLOR_BGR2GRAY)
p0=cv.goodFeaturesToTrack(old_gray,mask=None,**feature_params)
mask=np.zeros_like(old_frame)    # 이동 궤적을 저장할 영상

while(1):
    ret,frame=cap.read()
    if not ret: break
    new_gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    p1,match,err=cv.calcOpticalFlowPyrLK(old_gray,new_gray,p0,None,**lk_params)    # 광류 계산
    #                               이전 프레임 영상, 이전 프레임의 코너 특징점, 다음 프레임의 코너 특징점
    if p1 is not None:        
        # 양호한 쌍 선택
        good_new=p1[match==1]
        good_old=p0[match==1]
        
    # 이동 궤적 그리기
    for i in range(len(good_new)):
        a,b=int(good_new[i][0]),int(good_new[i][1])
        c,d=int(good_old[i][0]),int(good_old[i][1])
        mask=cv.line(mask,(a,b),(c,d),color[i].tolist(),2)
        frame=cv.circle(frame,(a,b),5,color[i].tolist(),-1)
       
    img=cv.add(frame,mask)

    cv.imshow('LTK tracker',img)
    cv.waitKey(30)

    old_gray=new_gray.copy()    
    p0=good_new.reshape(-1,1,2)
   
cv.destroyAllWindows()